# Линейная модель


На прошлом занятии познакомились с линейной моделью:
$$ \hat{y} = w_1 x_1 + w_2 x_2 + ... + w_k x_k + b,$$
где $\hat{y}$ - целевая переменная (что мы хотим предсказать), $x_j$ - $j$-ый признак объекта $x$, $w_j$ - вес $j$-го признака, $b$ - bias (смещение, свободный член), $k$ - количество признаков объектов. Иногда вместо $b$ используют обозначение $w_0$, а также вводят дополнительный фиктивный признак $x_0$, который всегда равен 1. (В такой модели будет $k+1$ признаков).

$$ \hat{y} = \sum\limits_{j=0}^k w_j x_j$$

В этом случае модель может быть записана так:
$$ \hat{y} = Xw,$$
$\hat{y}$ - вектор значений целевой переменной, $X$ - матрица значений признаков объектов, $w$ - вектор весов.

Важно отметить, что параметрами этой модели являются веса $w$. Когда говорят об обучении какого-либо алгоритма машинного обучения, как правило, имеют в виду настройку весов, т.е. параметров модели.  

Напоминим также, что в задаче линейной регрессии $\hat{y}$ - это действительное число. А можно ли линейную модель использовать для решения задани классификации? Да, конечно, можно. Например так:
$$ \hat{y} = sign(Xw),$$
где $sign$ функция, которая показывает знак выражения, записанного в скобках. И тогда $\hat{y} = \lbrace -1, 1 \rbrace$



# Предобработка признаков

Применяя линейную модель мы предполагаем, что каждый признак линейно влияет на значение целевой переменной. Такое предположение редко оказывается верным, поэтому нужно либо применять другую модель (то есть искать алгоритм в другом семействе алгоритмов), либо для улучшения качества модели необходимо специальным образом предобрабатывать признаки. Рассмотрим некоторые подходы для предобработки признаков (кстати, эти подходы могут улучшить и метрическую модель kNN).

Введём обозначения: $f$ - признак, $j$ - номер признака. Тогда $f_j (x)$ - это $j$-ый признак объекта.  





## Категориальный признак

Довольно частая ситуация, когда один из признаков принимает значения из некоего конечного множества.

Примеры: 
* в датасете sp500 у каждой компании указывался сектор: Industrials, Health Care, Information Technology, Financials и т.д.


Если $f_j (x)$ принимает значения из множества $C = \lbrace c_1, c_2, ..., c_m \rbrace$, то его можно заменить на m новых бинарных признаков, принимающих значения 1 для соответствующего значения и 0 для остальных. Такой подход называется **one-hot encoding**.

Самый быстрый способ реализации - `get_dummies` из библиотеки `pandas`. 

Существует также реализация `sklearn.preprocessing.OneHotEncoder`


In [ ]:
import pandas as pd

In [ ]:
sp500 = pd.read_csv(filepath_or_buffer = "sp500.csv",
                    sep = ',',
                    usecols=['Symbol', 'Sector', 'Price', 'Book Value'],
                    index_col='Symbol')

In [ ]:
sp500.head()

,Sector,Price,Book Value
Symbol,,,
MMM,Industrials,141.14,26.668
ABT,Health Care,39.60,15.573
ABBV,Health Care,53.95,2.954
ACN,Information Technology,79.79,8.326
ACE,Financials,102.91,86.897


In [ ]:
sectors = pd.get_dummies(sp500.Sector)
sectors

,Consumer Discretionary,Consumer Discretionary,Consumer Staples,Consumer Staples,Energy,Financials,Health Care,Industrials,Industries,Information Technology,Materials,Telecommunications Services,Utilities
Symbol,,,,,,,,,,,,,
MMM,0,0,0,0,0,0,0,1,0,0,0,0,0
ABT,0,0,0,0,0,0,1,0,0,0,0,0,0
ABBV,0,0,0,0,0,0,1,0,0,0,0,0,0
ACN,0,0,0,0,0,0,0,0,0,1,0,0,0
ACE,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YHOO,0,0,0,0,0,0,0,0,0,1,0,0,0
YUM,1,0,0,0,0,0,0,0,0,0,0,0,0
ZMH,0,0,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
sp500 = pd.concat([sp500, sectors], axis=1)
sp500.head()

,Sector,Price,Book Value,Consumer Discretionary,Consumer Discretionary,Consumer Staples,Consumer Staples,Energy,Financials,Health Care,Industrials,Industries,Information Technology,Materials,Telecommunications Services,Utilities
Symbol,,,,,,,,,,,,,,,,
MMM,Industrials,141.14,26.668,0,0,0,0,0,0,0,1,0,0,0,0,0
ABT,Health Care,39.60,15.573,0,0,0,0,0,0,1,0,0,0,0,0,0
ABBV,Health Care,53.95,2.954,0,0,0,0,0,0,1,0,0,0,0,0,0
ACN,Information Technology,79.79,8.326,0,0,0,0,0,0,0,0,0,1,0,0,0
ACE,Financials,102.91,86.897,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
del sp500['Sector']
sp500.head()

,Price,Book Value,Consumer Discretionary,Consumer Discretionary,Consumer Staples,Consumer Staples,Energy,Financials,Health Care,Industrials,Industries,Information Technology,Materials,Telecommunications Services,Utilities
Symbol,,,,,,,,,,,,,,,
MMM,141.14,26.668,0,0,0,0,0,0,0,1,0,0,0,0,0
ABT,39.60,15.573,0,0,0,0,0,0,1,0,0,0,0,0,0
ABBV,53.95,2.954,0,0,0,0,0,0,1,0,0,0,0,0,0
ACN,79.79,8.326,0,0,0,0,0,0,0,0,0,1,0,0,0
ACE,102.91,86.897,0,0,0,0,0,1,0,0,0,0,0,0,0


## Нелинейная зависимость переменной от признака

Довольно зависимость целевой переменной от значения признака является нелинейной. Для того, чтобы такой признак мог быть корректно использован в линейной модели, нам необходимо **бинаризовать признак**.

Бинаризация делается в два шага. На первом шаге определяем минимальное и максимальное значение признака. И разбиваем весь этот интервал на $m$ интервалов (чаще всего одинакового размера, но можно и похитрее). На втором шаге смотрим в какой интервал попадает значение и создаём $m$ новых бинарных признаков.

In [ ]:
## дополнительное задание: 
## в любом из рассмотренных ранее в курсе датасетов попробуйте реализовать бинаризацию признаков. 

Хорошая подборка разных способов предобработки признаков: https://nagornyy.me/courses/data-science/feature_engineering/

# Настройка параметров модели

На прошлом занятии упоминали способ вычислить значения параметров модели методом наименьших квадратов. Параметры эти могут быть найдены аналитически. Это решение наглядное, точное и короткое.
$$ w = (X^TX)^{-1}X^TY $$

Вот это  $(X^TX)^{-1}X^T$ называют псевдообратная матрица. Проблема метода заключается в том, что вычисление псевдообратной матрицы очень затратно, также аналитического решения может не оказаться вовсе. В связи с этим при настройке параметров модели сначала находят функционал качества (функция потерь), а затем ищут минимум функционал качества численными методами. 

## Эмпирический риск



Напомню про **минимизацию эмпирического риска**. Для задачи обучения по прецедентам вводится функция потерь: $L(\hat{y}, y)$, характеризующая величину отклонения ответа (прогноза, predict) модели $\hat{y} = a(x)$ от правильного ответа $y$ на одном произвольном объекте $x \in X$, где $a$ - это алгоритм, а $X$ - это множество объектов, имеющих признаковое описание, для которых известно значение целевой переменной. Пусть $\ell $ - это размерколичество таких объектов.

Эмпирический риск $Q$ - это функционал (то есть функция от функции) качества, характеризующий среднюю ошибку алгоритма на выборке:
$$ Q (a,X) = \frac{1}{\ell} \sum\limits_{i=1}^\ell L(\hat{y}, y) $$

Метод минимизация эмпирического риска заключается в том, чтобы в выбранном семействе алгоритмов найти такой алгоритм, который даст минимальное значение функционалу эмпирического риска. 

Таким образом, если мы выбираем в качестве семейства моделей линейный модели, а в качестве задачи - задачу регрессии, то это позволит нам подобрать наиболее подходящую функцию потерь $L(\hat{y}, y)$.

$$L(\hat{y}, y) = (\hat{y} - y)^2 $$

Обозначим $\hat{y}^{(i)}$ и $y^{(i)}$ соответственно прогноз и правильный ответ на  i-ом объекте $x^{(i)}$. Заметим, что $ \hat{y}^{(i)} = \sum\limits_{j=0}^k w_j x_j^{(i)} $. Тогда,

$$L(\hat{y}, y) = (\sum\limits_{j=0}^k w_j x_j^{(i)}  - y^{(i)})^2 $$

$$ Q (a,X) = \frac{1}{\ell} \sum\limits_{i=1}^\ell (\sum\limits_{j=0}^k w_j x_j^{(i)}  - y^{(i)})^2 $$

Итак, эта функция зависит от $\hat{y}$ и $y$, но на $y$ мы повлиять никак не можем. Когда мы будем искать минимум от эмпирического риска, мы можем повлиять только на $\hat{y}$.

При этом на $x \in X $ мы хоть и можем как-то повлиять, но мы сейчас рассматриваем тот момент, когда признаковое описание объектов уже сформировано. И тогда единственно, от чего будет реально зависеть функция потерь в момент нахождения минимума эмпирического риска, это $w$, то есть веса в модели.

$$ Q (w) = \frac{1}{\ell} \sum\limits_{i=1}^\ell (\sum\limits_{j=0}^k w_j x_j^{(i)}  - y^{(i)})^2 $$


## Градиентный спуск

Давайте попробуем реализовать алгоритм настройки параметров модели через производные.

Рассмотрим функцию $n$ переменных:
$Q(w) = Q(w_0, w_1, \dots, w_k)$.

Производная функции нескольких переменных по одной переменной называется **частной производной**.  **Градиент функции многих переменных** $\triangledown Q(w)$ - это вектор частных производных функции.

Производные функции от многих переменных можно определять, считая все аргументы, кроме того, по которому берем производную, константами. 

Известно, что градиент является направление наискорейшего роста функции, а антиградиент направлением наискорейшего убывания.

$$ \triangledown Q(w) = (\frac{\delta Q(w_0)}{\delta w_0}, \frac{\delta Q(w_1)}{\delta w_1}, \dots, \frac{\delta Q(w_k)}{\delta w_k} )$$

Идея алгоритма такая:
1. Фиксируем шаг обучения $\eta$ (например, $\eta = 0.001$). Шаг обучения - это гиперпараметр, мы его не обучаем, а подбираем.
2. Вычисляем градиент $\frac{\delta Q(w)}{\delta w}$
3. Для каждого измерения j изменяем $w_j$  по следующему правилу: $ w_j = w_j - \eta \frac{\delta Q(w_j)}{\delta w_j}$ 
4. Повторяем 3, пока не пройдет определенное количество шагов и/или мы не станем достаточно близко к точке минимума и/или значение $w_j$ перестанет изменяться.

Этот алгоритм называется **алгоритм градиентного спуска**. "Градиентного" -- потому что мы "спускаемся" к точке минимума, вычисляя производную (градиент) функции на каждом шаге.

Рассмотрим для случая с одним признаком.

$$ \triangledown Q(w) = (\frac{\delta Q(w_0)}{\delta w_0}, \frac{\delta Q(w_1)}{\delta w_1})$$


$$ \frac{ \delta Q(w_0)}{\delta w_0} = 2\sum\limits_{i=0}^\ell (w_0 + w_1x_1^{(i)}  - y^{(i)})x_0 = 2\sum\limits_{i=0}^\ell (w_0 + w_1x_1^{(i)}  - y^{(i)}) $$

$$ \frac{ \delta Q(w_1)}{\delta w_1} = 2\sum\limits_{i=0}^\ell (w_0 + w_1x_1^{(i)}  - y^{(i)})x_1 $$

Для поиска минимума константы перед суммой вообще никакой роли не играют, их можно не учитывать. 

Шаг (коэффициент, темп) обучения $\eta$  влияет на величину изменения веса на каждой эпохе.  Чем меньше шаг, тем выше точность, но тем больше итераций требуется для спуска в минимум. Но если сделать шаг слишком большим, то можно "перескочить" минимум, алгоритм может не сойтись. 

## Модификации градиентного спуска

Описанный выше стандартный подход иначе называется «пакетный» («batch»). При этом подходе для корректировки параметров модели на каждой итерации используется градиент, который считается как сумма градиентов, вызванных **каждым** элементом обучения. То есть стандартному градиентному спуску требуется проход по всем обучающим данным для того, чтобы изменить параметры на одной итерации. На больших наборах данных, это может быть не слишком эффективно. Поэтому существуют две модфикации:

* mini-batch градиентный спуск (на каждой итерации используем не все, а только небольшую часть объектов)
* cтохастический градиентныей спуск (на каждой итерации используем ровно один случайный объект).



Более подробно тут: https://www.machinelearningmastery.ru/gradient-descent-algorithm-and-its-variants-10f652806a3/

# Оценка качества моделей

## Метрики качества

 На прошлом занятии упоминали две метрики:
 * *mean_absolute_error* - средняя абсолютная ошибка $|y_i - \hat{y}_i|$
 * *mean_squared_error* - средняя квадратичная ошибка $(y_i - \hat{y}_i)^2$

Запишем формулу для средней квадратичной ошибки:

$$MSE =  \frac{1}{\ell} \sum\limits_{i=1}^\ell (\hat y^{(i)}  - y^{(i)})^2 $$

Для реальных задач чаще используют RMSE (квадратный корень из MSE), так как в MSE единицы измерения возводятся в квадрат. Например, когда мы предсказываем цену на квартиры в долларах, хотелось бы оценить в долларах (а не в квадратных долларах), на сколько мы ошибаемся.

Также используют метрику $R^2$ - коэффициент детерминации (он показывает отношение MSE к дисперсии).

Ещё один распространнённый вид ошибки средняя абсолютная процентная ошибка MAPE (и его модификация SMAPE).  

$$MAPE =  \frac{1}{\ell} \sum\limits_{i=1}^\ell (\hat y^{(i)}  - y^{(i)})/ y^{(i)}$$

$$SMAPE =  \frac{1}{\ell} \sum\limits_{i=1}^\ell \frac{(\hat y^{(i)}  - y^{(i)})} {(|\hat y^{(i)}| + |y^{(i)}|)/2}$$


## Кросс-валидация

Разделение набора данных на обучающую (train) и контрольную (test) - это стандартный подход для проверки **обобщающей способности алгоритма**. Этот подход называется подход с *отложенной выборкой*. При таком подходе мы обучаемся на одной выборке, а оцениваем качество на другой. Такой подход может привести к проблеме: наш результат будет зависеть от конкретного разбиения данных на обучающую и контролирующие выборки. 

Решается эта проблема с помощью **кросс-валидации**. Самый распространённый подход заключается в следующем: размеченные данные разбиваем на $k$ блоков (прошу прощения за путаницу, ранее у нас $k$ было количеством признаков, теперь мы эту же букву используем для другого, просто такой тип кросс-валидации по английски называется k-fold). Эти блоки имеют примерно одинаковый размер (наиболее распространённое количество блоков от 5 до 10). Обучаются k алгоритмов, при этом они обучаются на всех блоках, кроме одного. После каждая алгоритм оценивается на том блоке, который не участвовал в обучении. Оценка усредняется.



# Переобучение

Минимизация эмпирического риска - это универсальный  универсальный подход, позволяющий сводить задачу обучения к задачам численной оптимизации. Поэтому его используют повсеместно.

Основной недостаток этого подхода **эффект переобучения (overfitting)**, который возникает практически всегда при использовании метода минимизации эмпирического риска.

Основные способы борьбы с переобучением: ограничение сложности модели, в том числе отбор признаков, и наложение дополнительных ограничений на параметры модели, в том числе **регуляризация**.

Регуляризация - это подход для борьбы с переобучением, который заключается в изменении функционала качества путём добавления нового слагаемого, позволяющего учитывать величину весов. Рассмотрим несколько наиболее распространённых способов регуляризации.

Напомню функционал без штрафов за веса:
$$ Q (w) = \frac{1}{\ell} \sum\limits_{i=1}^\ell (\sum\limits_{j=0}^k w_j x_j^{(i)}  - y^{(i)})^2 $$



Добавим к этому ещё одно слагаемое $R(w)$

$$ Q (w) = \frac{1}{\ell} \sum\limits_{i=1}^\ell (\sum\limits_{j=0}^k w_j x_j^{(i)}  - y^{(i)})^2 + \alpha R(w)$$

$\alpha \geq  0$ - это гиперпараметр, определяющий степень влияния нового слагаемого.
Фактически же разные способы (виды) регуляризации задаются только различными видами $R(w)$

Наиболее распространены:

$$R(w) = \sum\limits_{j=0}^k |w_j|  $$ - лассо, L1 - регуляризация

$$R(w) = \sum\limits_{j=0}^k w_j^2  $$ - гребневая (ridge), L2 - регуляризация

Также существует способ ElasticNet, объединяющий в себе две перечисленные выше регуляризации
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet 

Фактически добаляются два новых слагаемых, а не одно.